# Latam Challenge

## Problema
El problema consiste en predecir la probabilidad de atraso de los vuelos que aterrizan o despegan del aeropuerto de Santiago de Chile (SCL). Para eso les entregamos un dataset usando datos públicos y reales donde cada fila corresponde a un vuelo que aterrizó o despegó de SCL. Para cada vuelo se cuenta con la siguiente información:
- Fecha-I : Fecha y hora programada del vuelo.
- Vlo-I : Número de vuelo programado.
- Ori-I : Código de ciudad de origen programado.
- Des-I : Código de ciudad de destino programado.
- Emp-I : Código aerolínea de vuelo programado.
- Fecha-O : Fecha y hora de operación del vuelo.
- Vlo-O : Número de vuelo de operación del vuelo.
- Ori-O : Código de ciudad de origen de operación
- Des-O : Código de ciudad de destino de operación.
- Emp-O : Código aerolínea de vuelo operado.
- DIA : Día del mes de operación del vuelo.
- MES : Número de mes de operación del vuelo.
- AÑO : Año de operación del vuelo.
- DIANOM : Día de la semana de operación del vuelo.
- TIPOVUELO : Tipo de vuelo, I =Internacional, N =Nacional.
- OPERA : Nombre de aerolínea que opera.
- SIGLAORI : Nombre ciudad origen.
- SIGLADES : Nombre ciudad destino

## Desafío
1. ¿Cómo se distribuyen los datos? ¿Qué te llama la atención o cuál es tu conclusión sobre esto?
2. Genera las columnas adicionales y luego expórtelas en un archivo synthetic_features.csv :
    - temporada_alta : 1 si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep, 0 si no.
    - dif_min : diferencia en minutos entre Fecha-O y Fecha-I .
    - atraso_15 : 1 si dif_min > 15, 0 si no.
    - periodo_dia : mañana (entre 5:00 y 11:59), tarde (entre 12:00 y 18:59) y noche (entre 19:00 y 4:59), en base a Fecha-I .
3. ¿Cómo se compone la tasa de atraso por destino, aerolínea, mes del año, día de la semana, temporada, tipo de vuelo?
¿Qué variables esperarías que más influyeran en predecir atrasos?
4. Entrena uno o varios modelos (usando el/los algoritmo(s) que prefieras) para estimar la probabilidad de atraso de un vuelo.
Siéntete libre de generar variables adicionales y/o complementar con variables externas.
5. Evalúa tu modelo. ¿Qué performance tiene? ¿Qué métricas usaste para evaluar esa performance y por qué? ¿Por qué
elegiste ese algoritmo en particular? ¿Qué variables son las que más influyen en la predicción? ¿Cómo podrías mejorar la
performance?

## Solución

In [1]:
# libraries
sys.path.append(r'..\common')
import sys
import pandas as pd
import numpy as np
import datetime as dt
# import common_helper as ch
from IPython.core.interactiveshell import InteractiveShell
from pandas_profiling import ProfileReport
from category_encoders import TargetEncoder
from ipywidgets import FloatProgress
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:,.2f}'.format

### 0. Lectura datos

In [2]:
%%time
dateparse = lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
schema = {
          'Fecha-I': object,
          'Vlo-I': str, 
          'Emp-I': str,
          'Des-I': str, 
          'Ori-I': str, #fijo
          'Fecha-O': object,
          'Vlo-O': str,
          'Ori-O': str, #fijo
          'Des-O': str, 
          'Emp-O':str, 
          'DIA': np.int64, 
          'MES': np.int64, 
          'AÑO': np.int64, 
          'DIANOM': str, 
          'TIPOVUELO': str,
          'OPERA': str, 
          'SIGLAORI': str, 
          'SIGLADES': str
          }
df = pd.read_csv(r'..\..\data\dataset_SCL.csv', dtype=schema, parse_dates=['Fecha-I', 'Fecha-O'], date_parser=dateparse)
df.columns = [name.lower().replace('-', '_').replace('ñ', 'ni') for name in df.columns]
df.head()

CPU times: total: 1.66 s
Wall time: 1.67 s


,fecha_i,vlo_i,ori_i,des_i,emp_i,fecha_o,vlo_o,ori_o,des_o,emp_o,dia,mes,anio,dianom,tipovuelo,opera,siglaori,siglades
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami


### 1. Distribución de los datos.

In [3]:
%%time
# Describe
df.describe(include='all', datetime_is_numeric=True)

CPU times: total: 109 ms
Wall time: 113 ms


,fecha_i,vlo_i,ori_i,des_i,emp_i,fecha_o,vlo_o,ori_o,des_o,emp_o,dia,mes,anio,dianom,tipovuelo,opera,siglaori,siglades
count,68206,68206,68206,68206,68206,68206,68205,68206,68206,68206,"68,206.00","68,206.00","68,206.00",68206,68206,68206,68206,68206
unique,NaN,584,1,64,30,NaN,861,1,63,32,NaN,NaN,NaN,7,2,23,1,62
top,NaN,174,SCEL,SCFA,LAN,NaN,174,SCEL,SCFA,LAN,NaN,NaN,NaN,Viernes,N,Grupo LATAM,Santiago,Buenos Aires
freq,NaN,686,68206,5787,37611,NaN,649,68206,5786,20988,NaN,NaN,NaN,10292,36966,40892,68206,6335
mean,2017-07-05 12:11:25.320059648,NaN,NaN,NaN,NaN,2017-07-05 12:20:31.971380992,NaN,NaN,NaN,NaN,15.71,6.62,"2,017.00",NaN,NaN,NaN,NaN,NaN
min,2017-01-01 00:15:00,NaN,NaN,NaN,NaN,2017-01-01 00:04:00,NaN,NaN,NaN,NaN,1.00,1.00,"2,017.00",NaN,NaN,NaN,NaN,NaN
25%,2017-03-31 12:41:15,NaN,NaN,NaN,NaN,2017-03-31 12:51:15,NaN,NaN,NaN,NaN,8.00,3.00,"2,017.00",NaN,NaN,NaN,NaN,NaN
50%,2017-07-10 05:30:00,NaN,NaN,NaN,NaN,2017-07-10 05:37:00,NaN,NaN,NaN,NaN,16.00,7.00,"2,017.00",NaN,NaN,NaN,NaN,NaN
75%,2017-10-08 11:18:45,NaN,NaN,NaN,NaN,2017-10-08 11:19:45,NaN,NaN,NaN,NaN,23.00,10.00,"2,017.00",NaN,NaN,NaN,NaN,NaN
max,2017-12-31 23:55:00,NaN,NaN,NaN,NaN,2018-01-01 00:12:00,NaN,NaN,NaN,NaN,31.00,12.00,"2,018.00",NaN,NaN,NaN,NaN,NaN


In [6]:
%%time
# pandas profiling
# Discard columns=['vlo_i', 'ori_i', 'ori_o', 'siglaori', 'anio'] because they are not useful
profile = ProfileReport(df.drop(columns=['vlo_i', 'ori_i', 'ori_o', 'siglaori', 'anio']), title="Profiling Report")

CPU times: total: 15.6 ms
Wall time: 15.4 ms


In [7]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

**Conclusiones**:

- Dado que los campos de fecha los datos ``ori_i``  ``ori_o`` y ``siglaori`` y `anio` solo tienen valores constantes se excluyen del profiling. 
- El campo ``vlo_i`` tambièn se excluye dado que es un identificador que no aporta información.
- Sobre los dos campos de fecha ``fecha_i`` y ``fecha_o`` se observan picos sobre ciertas temporadas (Tambièn se observa con la variable ``mes``)
    * De enero, hasta comienzos de marzo.
    * A mediados de Julio hay un pequeño pico, donde a partir de agosto el número de vuelos aumenta hasta diciembre.
- El campo ```día``` se observa con una baja sobre los ultimos días(Asociado posiblemente a que algunos meses no tienen los 31 días)
- Alta cardinalidad sobre algunas de las variables categoricas, asi como altas correlaciones entre las mismas.


### 2. Generación de Columnas Adicionales.
Genera las columnas adicionales y luego expórtelas en un archivo synthetic_features.csv :
- temporada_alta : 1 si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep, 0 si no.
- dif_min : diferencia en minutos entre Fecha-O y Fecha-I .
- atraso_15 : 1 si dif_min > 15, 0 si no.
- periodo_dia : mañana (entre 5:00 y 11:59), tarde (entre 12:00 y 18:59) y noche (entre 19:00 y 4:59), en base a
Fecha-I .

### 3. Composición del atraso.

### 4. Entrenamiento del modelo.

### 5. Evaluación del modelo.